In [80]:
import numpy as np
import rawpy


def pack_raw(raw, white_level):
    """Packs Bayer image to 4 channels"""
    img = raw.raw_image_visible


    img = np.expand_dims(img, axis=2)

    out = np.concatenate((img[0::2, 0::2],
                          img[0::2, 1::2],
                          img[1::2, 1::2],
                          img[1::2, 0::2]), axis=2, dtype=np.float32)
    black_level = np.array(raw.black_level_per_channel, dtype=np.float32)[None, None, :]
    
    print(out.max(), out.min(), white_level, black_level)
    out = (out - black_level) / (float(white_level) - black_level)
    print(out.max(), out.min())


    out = (out * 255.0).astype(np.int8)
    return out

In [81]:
import cv2


def align_images_single_channel(img1, img2):
    """Aligns img2 to img1 using ORB feature matching and RANSAC."""
    orb = cv2.ORB_create(nfeatures=5000)

    # Detect keypoints and descriptors
    kp1, des1 = orb.detectAndCompute(img1, mask=None)
    kp2, des2 = orb.detectAndCompute(img2, mask=None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Use RANSAC to find homography
    if len(matches) > 10:
        src_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold=5.0)
        
        # Warp img2 to align with img1 (preserving colors)
        aligned_img = cv2.warpPerspective(img2, H, (img1.shape[1], img1.shape[0]))
        
        return aligned_img, H
    else:
        raise ValueError("Not enough matches found.")


In [ ]:
original_image_raw = rawpy.imread('first-dataset/first-dataset-RAW/IMG_7781.CR2')
diffused_image_raw = rawpy.imread('first-dataset/first-dataset-RAW/IMG_7782.CR2')

packed_original = pack_raw(original_image_raw, original_image_raw.white_level)
packed_diffused = pack_raw(diffused_image_raw, original_image_raw.white_level)



#print(packed_original.max(), packed_original.min())
#align_images_single_channel(
#   packed_original[:, :, 0][:, :, None],
#   packed_diffused
#   #np.repeat(packed_original[:, :, 0][:, :, None], repeats=3, axis=2),
#    #np.repeat(packed_diffused[:, :, 0][:, :, None], repeats=3, axis=2),
#)
original_image_raw.camera_white_level_per_channel, original_image_raw.white_level


16088.0 1060.0 14605 [[[1024. 1024. 1024. 1025.]]]
1.1091967 0.0026507622
16088.0 1016.0 14605 [[[1024. 1024. 1024. 1024.]]]
1.1091967 -0.00058905827


([15319, 15319, 15319, 15319], 14605)

In [33]:
np.repeat(packed_original[:, :, 0][:, :, None], repeats=3, axis=2)

array([[[ 5,  5,  5],
        [ 5,  5,  5],
        [ 5,  5,  5],
        ...,
        [46, 46, 46],
        [46, 46, 46],
        [47, 47, 47]],

       [[ 5,  5,  5],
        [ 5,  5,  5],
        [ 5,  5,  5],
        ...,
        [47, 47, 47],
        [48, 48, 48],
        [48, 48, 48]],

       [[ 5,  5,  5],
        [ 5,  5,  5],
        [ 6,  6,  6],
        ...,
        [47, 47, 47],
        [46, 46, 46],
        [48, 48, 48]],

       ...,

       [[13, 13, 13],
        [15, 15, 15],
        [13, 13, 13],
        ...,
        [61, 61, 61],
        [60, 60, 60],
        [63, 63, 63]],

       [[13, 13, 13],
        [11, 11, 11],
        [10, 10, 10],
        ...,
        [63, 63, 63],
        [63, 63, 63],
        [59, 59, 59]],

       [[ 7,  7,  7],
        [ 7,  7,  7],
        [ 7,  7,  7],
        ...,
        [59, 59, 59],
        [61, 61, 61],
        [60, 60, 60]]], dtype=int8)

In [22]:

def align_images_raw(path1, path2):
    raw1 = rawpy.imread(path1)
    raw2 = rawpy.imread(path2)

    packed1 = pack_raw(raw1, raw1.white_level)    

    packed1R, packed1G1, packed1B, packed1G2 = packed1[:,:,0], packed1[:,:,1], packed1[:,:,2], packed1[:,:,3]
    
    packed2 = pack_raw(raw2, raw1.white_level)

    packed2R, packed2G1, packed2B, packed2G2 = packed2[:,:,0], packed2[:,:,1], packed2[:,:,2], packed2[:,:,3]

    pairs = [(packed1R, packed2R), (packed1G1, packed2G1), (packed1B, packed2B), (packed1G2, packed2G2)]
    aligned_images = []
    projection_matrices = []
    
    # Calculate projections for each color channel individually
    for img1, img2 in pairs:
        aligned, projection_matrix = align_images_single_channel(img1[:,:,None], img2[:,:,None])
        aligned = aligned
        aligned_images.append(aligned)
        projection_matrices.append(projection_matrix)

    # TODO: check projection matrices

    H = 2 * packed1.shape[0]
    W = 2 * packed1.shape[1]

    
    raw = rawpy.imread(path1)
    raw.raw_image_visible[0:H:2, 0:W:2] = aligned_images[0]
    raw.raw_image_visible[0:H:2, 1:W:2] = aligned_images[1]
    raw.raw_image_visible[1:H:2, 1:W:2] = aligned_images[2]
    raw.raw_image_visible[1:H:2, 0:W:2] = aligned_images[3]
    #TODO: crop original image

    #TODO: intensity alignemnt

    return raw

In [23]:
align_images_raw('first-dataset/first-dataset-RAW/IMG_7781.CR2', 'first-dataset/first-dataset-RAW/IMG_7782.CR2')

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<1, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>, cv::impl::(anonymous namespace)::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<1, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
